In [1]:
import numpy

from cicada.additive import AdditiveProtocolSuite
from cicada.communicator import SocketCommunicator
from cicada.encoding import FixedPoint

def main(communicator):
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    c_share = protocol.add(a_share, b_share)
    c = protocol.reveal(c_share)

Note that we're using a very small field (`order=127`) and a very low-precision encoding (`precision=2`) to make the values in the logged output easier to read.

In [2]:
import logging

from cicada import transcript

with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.net_handler())
    SocketCommunicator.run(fn=main, world_size=3);

Player 1: --> 2 PRZS 3988476335160044916
Player 0: --> 1 PRZS 3522766875086706026
Player 2: --> 0 PRZS 7928856463745416826
Player 1: <-- 0 PRZS 3522766875086706026
Player 2: <-- 1 PRZS 3988476335160044916
Player 0: <-- 2 PRZS 7928856463745416826
Player 2: --> 0 GATHER 88
Player 1: --> 0 GATHER 7
Player 0: --> 0 GATHER 52
Player 2: --> 1 GATHER 88
Player 1: --> 1 GATHER 7
Player 0: <-- 2 GATHER 88
Player 1: <-- 1 GATHER 7
Player 2: --> 2 GATHER 88
Player 0: <-- 1 GATHER 7
Player 1: <-- 2 GATHER 88
Player 2: <-- 2 GATHER 88
Player 0: <-- 0 GATHER 52
Player 0: --> 1 GATHER 52
Player 0: --> 2 GATHER 52
Player 1: <-- 0 GATHER 52
Player 2: <-- 0 GATHER 52
Player 1: --> 2 GATHER 7
Player 2: <-- 1 GATHER 7


In [3]:
def main(communicator):
    transcript.log("Let's setup additive sharing!")
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.net_handler())
    SocketCommunicator.run(fn=main, world_size=3);

Player 0: Let's setup additive sharing!
Player 1: Let's setup additive sharing!
Player 2: Let's setup additive sharing!
Player 0: --> 1 PRZS 912044508948196820
Player 2: --> 0 PRZS 3046225897000249346
Player 1: --> 2 PRZS 6751254543502116700
Player 0: <-- 2 PRZS 3046225897000249346
Player 2: <-- 1 PRZS 6751254543502116700
Player 1: <-- 0 PRZS 912044508948196820
Player 1: Let's share some secrets!
Player 0: Let's share some secrets!
Player 2: Let's share some secrets!
Player 2: Let's add some secrets!
Player 1: Let's add some secrets!
Player 0: Let's add some secrets!
Player 2: Let's reveal the results!
Player 1: Let's reveal the results!
Player 0: Let's reveal the results!
Player 2: --> 0 GATHER 69
Player 1: --> 0 GATHER 64
Player 0: --> 0 GATHER 14
Player 2: --> 1 GATHER 69
Player 1: --> 1 GATHER 64
Player 0: <-- 2 GATHER 69
Player 2: --> 2 GATHER 69
Player 1: <-- 2 GATHER 69
Player 0: <-- 0 GATHER 14
Player 2: <-- 2 GATHER 69
Player 1: <-- 1 GATHER 64
Player 0: <-- 1 GATHER 64
Player

In [4]:
with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.net_handler(fmt="# {processName}: {msg}", netfmt="{processName},{net.verb},{net.src},{net.dst},{net.tag},{net.payload}"))
    SocketCommunicator.run(fn=main, world_size=3);

# Player 0: Let's setup additive sharing!
# Player 2: Let's setup additive sharing!
# Player 1: Let's setup additive sharing!
Player 1,sent,1,2,PRZS,1181905445099100775
Player 0,sent,0,1,PRZS,1967890497715291908
Player 2,sent,2,0,PRZS,3336992880123390415
Player 2,received,1,2,PRZS,1181905445099100775
Player 1,received,0,1,PRZS,1967890497715291908
Player 0,received,2,0,PRZS,3336992880123390415
# Player 2: Let's share some secrets!
# Player 0: Let's share some secrets!
# Player 1: Let's share some secrets!
# Player 2: Let's add some secrets!
# Player 0: Let's add some secrets!
# Player 1: Let's add some secrets!
# Player 2: Let's reveal the results!
# Player 0: Let's reveal the results!
Player 2,sent,2,0,GATHER,49
# Player 1: Let's reveal the results!
Player 2,sent,2,1,GATHER,49
Player 0,sent,0,0,GATHER,19
Player 1,sent,1,0,GATHER,79
Player 2,sent,2,2,GATHER,49
Player 0,received,2,0,GATHER,49
Player 1,received,2,1,GATHER,49
Player 2,received,2,2,GATHER,49
Player 0,received,0,0,GATHER,19


If you look carefully, you can see that this produces two nearly identical events for each message (once when the message is sent, and once when the message is received).  If you wish to eliminate the duplication, e.g. by only logging messages when they're sent, you can specify that too:

In [5]:
with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.net_handler(received=False, fmt="# {processName}: {msg}", netfmt="{processName},{net.verb},{net.src},{net.dst},{net.tag},{net.payload}"))
    SocketCommunicator.run(fn=main, world_size=3);

# Player 0: Let's setup additive sharing!
# Player 1: Let's setup additive sharing!
# Player 2: Let's setup additive sharing!
Player 0,sent,0,1,PRZS,469539288949084103
Player 1,sent,1,2,PRZS,3984976033566602272
Player 2,sent,2,0,PRZS,1976810725153270119
# Player 1: Let's share some secrets!
# Player 2: Let's share some secrets!
# Player 0: Let's share some secrets!
# Player 2: Let's add some secrets!
# Player 1: Let's add some secrets!
# Player 0: Let's add some secrets!
# Player 2: Let's reveal the results!
# Player 1: Let's reveal the results!
# Player 0: Let's reveal the results!
Player 2,sent,2,0,GATHER,21
Player 1,sent,1,0,GATHER,88
Player 0,sent,0,0,GATHER,38
Player 2,sent,2,1,GATHER,21
Player 1,sent,1,1,GATHER,88
Player 2,sent,2,2,GATHER,21
Player 0,sent,0,1,GATHER,38
Player 0,sent,0,2,GATHER,38
Player 1,sent,1,2,GATHER,88


In [6]:
from cicada.arithmetic import Field

with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.code_handler())
    
    f = Field(order=127)
    a = f.ones(3)
    b = f.uniform(size=3, generator=numpy.random.default_rng())
    f.inplace_add(a, b)

cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).ones(shape=3), numpy.array([1, 1, 1], dtype=object))

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 128070599164706723044894626344546904748, 'inc': 336130079396890558782550967416732276033}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=3, generator=numpy.random.Generator(bg)), numpy.array([138, 253, 128], dtype=object))

lhs = numpy.array([1, 1, 1], dtype=object)
cicada.arithmetic.Field(order=127).inplace_add(lhs=lhs, rhs=numpy.array([138, 253, 128], dtype=object))
cicada.transcript.assert_equal(lhs, numpy.array([12, 0, 2], dtype=object))

